In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json

In [2]:
import dataset
import algorithm

In [3]:
CONFIG = json.load(open('config.json'))

In [4]:
Dataset = dataset.download_dataset(CONFIG['data']['filename'])

In [5]:
Dataset.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida_piso_2,...,banios,banios_sin_ducha,dormitorios,cocinas,chimeneas,area_garage,area_piscina,mes_venta,anio_venta,precio
0,8450,7,5,2003,2003,Gd,GasA,Y,856,854,...,2,1,3,1,0,548,0,2,2008,208500
1,9600,6,8,1976,1976,Gd,GasA,Y,1262,0,...,2,0,3,1,1,460,0,5,2007,181500
2,11250,7,5,2001,2002,Gd,GasA,Y,920,866,...,2,1,3,1,1,608,0,9,2008,223500
3,9550,7,5,1915,1970,TA,GasA,Y,961,756,...,1,0,3,1,1,642,0,2,2006,140000
4,14260,8,5,2000,2000,Gd,GasA,Y,1145,1053,...,2,1,4,1,1,836,0,12,2008,250000


In [6]:
data = dataset.get_data(Dataset)
data.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida_piso_2,...,banios,banios_sin_ducha,dormitorios,cocinas,chimeneas,area_garage,area_piscina,mes_venta,anio_venta,precio
0,8450,7,5,2003,2003,0,1,1,856,854,...,2,1,3,1,0,548,0,2,2008,208500
1,9600,6,8,1976,1976,0,1,1,1262,0,...,2,0,3,1,1,460,0,5,2007,181500
2,11250,7,5,2001,2002,0,1,1,920,866,...,2,1,3,1,1,608,0,9,2008,223500
3,9550,7,5,1915,1970,1,1,1,961,756,...,1,0,3,1,1,642,0,2,2006,140000
4,14260,8,5,2000,2000,0,1,1,1145,1053,...,2,1,4,1,1,836,0,12,2008,250000


In [7]:
normalize_data = dataset.Normalizer.transform(data)

In [8]:
data_x, data_y = dataset.split_data_x_y(normalize_data)

In [9]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [10]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [11]:
# DataLoader => Devuelve un conjunto de iteraciones, cada iteracion contiene un arreglo de 'batch_size' registros
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [12]:
model = algorithm.Model(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [13]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    loss_train, lr_train = model.run_epoch(train_dataloader, is_training=True)
    loss_test, lr_test = model.run_epoch(test_dataloader)
    if epoch == 0 or ((epoch+1)%10 == 0):
        print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f} | lr:{:.6f}'
              .format(epoch+1, CONFIG["training"]["num_epoch"], loss_train, loss_test, lr_train))

Epoch[1/100] | loss train:0.008330, test:0.001252 | lr:0.010000
Epoch[10/100] | loss train:0.001127, test:0.000364 | lr:0.010000
Epoch[20/100] | loss train:0.000868, test:0.000444 | lr:0.010000
Epoch[30/100] | loss train:0.000839, test:0.000344 | lr:0.010000
Epoch[40/100] | loss train:0.000474, test:0.000256 | lr:0.010000
Epoch[50/100] | loss train:0.000500, test:0.000361 | lr:0.010000
Epoch[60/100] | loss train:0.000337, test:0.000267 | lr:0.010000
Epoch[70/100] | loss train:0.000398, test:0.000279 | lr:0.010000
Epoch[80/100] | loss train:0.000311, test:0.000234 | lr:0.010000
Epoch[90/100] | loss train:0.000251, test:0.000248 | lr:0.010000
Epoch[100/100] | loss train:0.000284, test:0.000368 | lr:0.010000


In [14]:
# Verificacion de la prediccion del modelo
for idx, (x, y) in enumerate(test_dataloader):
    x = x.to(CONFIG["training"]["device"])
    out = model(x)
    out_vector = out.detach().numpy()
    y_vector = y.numpy()
    
    for i in range(len(y_vector)):
        price_predict = dataset.Normalizer.inverse_transform(out_vector[i], 21)
        price_real = dataset.Normalizer.inverse_transform(y_vector[i], 21)
        print('Price predict: {0}, Price real: {1}, Diferencia: {2}'.format(price_predict, price_real, price_predict-price_real))
        
    break

Price predict: 114752, Price real: 107000, Diferencia: 7752
Price predict: 191051, Price real: 175500, Diferencia: 15551
Price predict: 144790, Price real: 137000, Diferencia: 7790
Price predict: 274485, Price real: 227000, Diferencia: 47485
Price predict: 201597, Price real: 179600, Diferencia: 21997
Price predict: 176609, Price real: 162900, Diferencia: 13709
Price predict: 138110, Price real: 158000, Diferencia: -19890
Price predict: 204309, Price real: 200000, Diferencia: 4309
Price predict: 148138, Price real: 129500, Diferencia: 18638
Price predict: 81588, Price real: 52500, Diferencia: 29088
Price predict: 179187, Price real: 184000, Diferencia: -4813
Price predict: 149351, Price real: 163000, Diferencia: -13649
Price predict: 139296, Price real: 143750, Diferencia: -4454
Price predict: 166716, Price real: 177000, Diferencia: -10284
Price predict: 122942, Price real: 119000, Diferencia: 3942
Price predict: 111455, Price real: 119500, Diferencia: -8045
Price predict: 90871, Price